In [4]:
lista = 2
if type(lista) != list:
    print('não é uma lista')

não é uma lista


In [7]:
dicio = {'teste1':1,'teste2':2}
for name, value in dicio.items():
    print(name,value)

teste1 1
teste2 2


In [29]:
import sympy as sp

x,p,A = sp.symbols('x,p,A')

f = sp.Function('f')(x)
#A = sp.Function('A')(x)
N = sp.Function('N')(x,p)

expr = sp.Eq(f.diff(x),f)
expr

Eq(Derivative(f(x), x), f(x))

In [31]:
expr.subs(f,A + x*N)

Eq(Derivative(A + x*N(x, p), x), A + x*N(x, p))

In [33]:
print(sp.simplify(expr.subs(f,A + x*N)))

Eq(A + x*N(x, p), x*Derivative(N(x, p), x) + N(x, p))


In [56]:
expr2 = sp.Eq(x**2,0)
expr2.lhs -expr2.rhs

x**2